things to explain - backward induction for american option pricing

`numpy` things to know for the paper:
- `np.polyfit`
- `np.ploy1d`
- `np.polyval`
- `np.where`
- `np.random.standard_normal`
- `np.cumsum`

In [1]:
# imports
import numpy as np

In [2]:
temp = np.random.standard_normal((5,3))
print(temp)
print('')
print(np.cumsum(temp, axis=0))
print('')
print(np.cumsum(temp, axis=1))

[[-2.65674301  0.88775234  2.11675046]
 [-0.58563122  0.23118172  1.05679602]
 [ 1.38312036  0.256484   -0.97015603]
 [ 0.84979421 -1.1450417  -0.71711014]
 [ 0.98075995  1.58980443 -1.1210115 ]]

[[-2.65674301  0.88775234  2.11675046]
 [-3.24237424  1.11893406  3.17354648]
 [-1.85925387  1.37541806  2.20339044]
 [-1.00945966  0.23037637  1.48628031]
 [-0.02869971  1.82018079  0.36526881]]

[[-2.65674301 -1.76899067  0.34775979]
 [-0.58563122 -0.35444951  0.70234651]
 [ 1.38312036  1.63960437  0.66944833]
 [ 0.84979421 -0.29524748 -1.01235762]
 [ 0.98075995  2.57056438  1.44955288]]


In the european option, only the terminal point is important.

In American options, the past is important, because the option can be exercised before the terminal point. Therefore we need to generate a matrix of random numbers and compute the cumulative sum because the stock price at time t is the cumulative sum of the previous stock returns.

In [3]:
# Longstaff & Schwartz algorithm
import math
import numpy as np
from math import exp, sqrt, log
np.random.seed(123456)

# input specifications
S0 = 100.         # underlying price (S_0)
sigma = 0.20978   # volatility per year
r = 0.01          # risk free rate per year
div = 0.001       # dividend yield per year
mu = r - div      # expected returns under risk neutral measure
T = 1.            # time to maturity
K = 110.          # exercise price

# Simulation parameters
I = 10**3                 # number of simulations
M = 12                    # time steps (unit is years)
dt = T/M                  # length of time steps
df = math.exp(-r * dt)    # discount factor

In [4]:
return_series = (mu - (sigma**2)/2.) * dt + sigma * sqrt(dt)* np.random.standard_normal((M, I))
S = S0 * np.exp(np.cumsum(return_series, axis = 0))
print(S.shape)
# note: shape is (M, I)
# print(S)

(12, 1000)


In [6]:
# compute inner value ( inner value == the value of the option if it is exercised right away )
h = np.maximum(K - S, 0.) # exercising right away
V = h[-1] # terminal value

#  Let us compute the value of american option with backward induction 
polyfit_order = 10
for t in range(M-1, 0, -1):
    rg = np.polyfit(S[t], V*df, polyfit_order) # estimate continuation-value function with stock prices
    C = np.polyval(rg, S[t]) # evaluate continuation value with stock prices with function
    V = np.where(h[t]>C, h[t], V*df) # update american value
    
V0 = df * np.average(V) # finally, compute the value of the american option 
print(f"The american option value is: {round(V0,4)}")

The american option value is: 14.352
